In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import tensorflow.contrib.slim as slim
from nets.vgg import vgg_16 as vgg
from nets.vgg import vgg_arg_scope
import preprocessing.imagenet_utils as imagenet_utils
import preprocessing.vgg as preprocessing
import PIL.Image as Image
import glob

In [ ]:

def load_image(img_path):
    s_image = (Image.open(os.path.join(img_path)).convert('RGB'))
    s_image = s_image.resize([224, 224], resample=Image.BICUBIC)
    re_image = s_image
    s_image = np.asarray(s_image, dtype=np.float32)
    s_image = preprocessing.mean_image_subtraction(s_image)
    s_image = np.expand_dims(s_image, axis=0)
    return s_image, re_image


model_path = './modelparams/vgg_16.ckpt'
labels_to_names, label_names = imagenet_utils.create_readable_names_for_imagenet_labels()

cates = ['clock' ,'crib','fan','garbage_can','microwave_oven','rug','socks','teddy_bear','tent']

for c in range(len(cates)):
    image_dir = "./object_array_displays_fixation_data/training_image_sets/" + cates[c]
    image_paths = [glob.glob(image_dir + "/training_set_IN/*.JPEG")]

    sample_size = len(image_paths[0])
    print(sample_size)
    conv5_3_features = np.zeros((sample_size,14,14,512),dtype=float)

    g_2 = tf.Graph()          
    with g_2.as_default():
        vgg_inputs = tf.placeholder(dtype=tf.float32, shape=[None, 224, 224, 3], name='input_image')
        with slim.arg_scope(vgg_arg_scope()):
            vgg_output, vgg_endpoints = vgg(vgg_inputs, is_training=False)

        init_fn = slim.assign_from_checkpoint_fn(model_path, slim.get_model_variables('vgg_16'))

        sess_2 = tf.Session(graph=g_2)
        sess_2.run(tf.global_variables_initializer())
        init_fn(sess_2)
        print('Running the session')
        for image_index in range(sample_size):
            image_path = image_paths[0][image_index]
#             print(image_path)
#             print(image_index)
            im_name, ext = image_path.split('/')[-1].split('.')

            x_im, img = load_image(image_path)
#             plt.imshow(img)
#             plt.show()
            vgg_all_output = sess_2.run([vgg_output, vgg_endpoints], feed_dict={vgg_inputs: x_im})
            [current_vgg_output, current_vgg_endpoints] = vgg_all_output 
            conv5_3_feat = current_vgg_endpoints['vgg_16/conv5/conv5_3']   # ['vgg_16/pool3']
            conv5_3_features[image_index,:,:,:] = conv5_3_feat[0]


    #         for i, catId in enumerate(selected_cates):
    #             # !!!: Notice here this is a catId+1 instead of catID
    #             print('{:d}\t category: {:s}; confidence:{:.4f},\tlabelID: {:s}'.format(i, labels_to_names[
    #                 catId + 1], s_output[catId], label_names[catId]))

    print(cates[c])  
    features_file_name = image_dir +'/'+ cates[c] + '_conv5_3.npy'
    np.save(features_file_name, conv5_3_features)
    
    w_file_name = image_dir +'/'+ cates[c] + '_conv5_3_max_map_w.npy'
    ccf_im_max = np.max(np.reshape(conv5_3_features,(sample_size,14*14,512)), axis=1)
    np.save(w_file_name, np.mean(ccf_im_max , axis=0) )
    
    w_file_name = image_dir +'/'+ cates[c] + '_conv5_3_mean_map_w.npy'
    ccf_im_mean = np.mean(np.reshape(conv5_3_features,(sample_size,14*14,512)), axis=1)
    np.save(w_file_name, np.mean(ccf_im_mean , axis=0) )
    
    print("max map")
    plt.imshow(ccf_im_max)
    plt.savefig(image_dir +'/'+ cates[c] + '_conv5_3_max_map_ccf.jpg')
    plt.show()
    print("mean map")
    plt.imshow(ccf_im_mean)
    plt.savefig(image_dir +'/'+ cates[c] + '_conv5_3_mean_map_ccf.jpg')
    plt.show()